# Imperative Event Handling: The Observer Pattern


We are going to look at traditional ways to handle event handling. The traditional way to deal with user interface is with observer pattern. We are going to look at different way to treat events in these programs functional reactive programming where events are essentially summarized as signals.

The Observer Pattern is widely used when views need to react to changes in a model.
Some variants are also called 
* publish/subscribe  
* model/view/controller (MVC)

The idea is we have some sort of model, which captures the state of an appplication. We might have one or more views that present the state. Views can announce themselves to the model with a method called subscribe. The model will then change publish the fact new information is available. Views can enquire the model and change it's presentation. There could be multiple views subscribed to the same model. 

Here is a trait for publisher.

In [1]:
trait Publisher {
private var subscribers: Set[Subscriber] = Set()
def subscribe(subscriber: Subscriber): Unit =
subscribers += subscriber
def unsubscribe(subscriber: Subscriber): Unit =
subscribers -= subscriber
def publish(): Unit =
subscribers.foreach(_.handler(this))
}
trait Subscriber {
def handler(pub: Publisher)
    
}

defined trait Publisher
defined trait Subscriber

Publishers maintain internally a set of subscribers. You can add a new subsriber to the publisher using method `subscribe`, which announces the publisher. The dual of `subscribe` is `unsubscribe`. Anothe method `publish`, it would go through all subscribers and invokes `handler` method.

Subscriber need to have only `handler` method. We pass the publisher that publish new information as parameter.

Let’s make BankAccount a Publisher:

In [2]:
class BankAccount extends Publisher {
  //we publish everytime we change the state of the bank account
  private var balance = 0
  
  //accessor method
  def currentBalance = balance       
  
  def deposit(amount: Int): Unit = {
    if (amount > 0) balance = balance + amount
    publish()            // <--
  }
  
  def withdraw(amount: Int): Unit = {
    if (0 < amount && amount <= balance) {
      balance = balance - amount
      publish()          // <--
    } else throw new Error("insufficient funds")
  }

}


defined class BankAccount

A Subscriber to maintain the total balance of a list of accounts:

In [3]:
//consolidator is a subscriber to maintain the total balance of a list of account
class Consolidator(observed: List[BankAccount]) extends Subscriber {
  observed.foreach(_.subscribe(this)) //subscribe to all BankAccounts
  
  private var total: Int = _
  compute()
  
  private def compute() = {
    total = observed.map(_.currentBalance).sum
  }
  //everytime something changes, you compute
  def handler(pub: Publisher) = compute()
  def totalBalance = total
}

defined class Consolidator

`Consolidator` is a subscriber. It maintians total balance of all accounts. Whenver a bankaccount changed using `handler` method, the results are published (using `publish` method in Publisher), by calling `sum` method, which updates the total balance.
One could envision many ways to do this. Finally `totalBalance` accesor methods gives total balance.

In [4]:
val a = new BankAccount()
val b = new BankAccount()
val c = new Consolidator(List(a,b))

a: BankAccount = ammonite.$sess.cmd1$Helper$BankAccount@16bbbe2
b: BankAccount = ammonite.$sess.cmd1$Helper$BankAccount@97b0c5
c: Consolidator = ammonite.$sess.cmd2$Helper$Consolidator@b5cc3e

In [5]:
c.totalBalance

res4: Int = 0

In [6]:
a deposit 20


In [7]:
c.totalBalance

res6: Int = 20

In [8]:
b deposit 30

In [9]:
c.totalBalance

res8: Int = 50

## Observer Pattern, The Good
* Decouples views from state
* Allows to have a varying number of views of a given state
* Simple to set up

## Observer Pattern, The Bad
- Forces imperative style, since handlers are Unit-typed
- Many moving parts that need to be co-ordinated (every publisher has to announce and every subscriber handler need to be called)
- Concurrency makes things more complicated
- Views are still tightly bound to one state; view update happens immediately.
